In [ ]:
import pandas as pd
train = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/train.csv')

In [ ]:
import os
path = '/kaggle/input/cassava-leaf-disease-classification/'
for i in os.listdir(path):
    print(i)

In [ ]:
train

In [ ]:
train.info()

In [ ]:
train['label'].unique()

In [ ]:
from PIL import Image  
im = Image.open("/kaggle/input/cassava-leaf-disease-classification/train_images/999616605.jpg")  
print('opened')
im.size

In [ ]:
train_path = '/kaggle/input/cassava-leaf-disease-classification/train_images'
test_path = '/kaggle/input/cassava-leaf-disease-classification/test_images'

In [ ]:
import json

file = open('/kaggle/input/cassava-leaf-disease-classification/label_num_to_disease_map.json')
json_data = json.load(file)

In [ ]:
json_data

In [ ]:
for i in range(21397):
    if train['label'][i] == 0:
        train.replace(to_replace =train['label'][i], 
                 value ='Cassava Bacterial Blight (CBB)',inplace=True) 
    elif train['label'][i] == 1:
        train.replace(to_replace =train['label'][i], 
                 value ='Cassava Brown Streak Disease (CBSD)',inplace=True) 
    elif train['label'][i] == 2:
        train.replace(to_replace =train['label'][i], 
                 value ='Cassava Green Mottle (CGM)',inplace=True) 
    elif train['label'][i] == 3:
        train.replace(to_replace =train['label'][i], 
                 value ='Cassava Mosaic Disease (CMD)',inplace=True) 
    elif train['label'][i] == 4:
        train.replace(to_replace =train['label'][i], 
                 value ='Healthy',inplace=True) 
        

In [ ]:
train.head()

In [ ]:
import keras
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import * 
from tensorflow.keras.preprocessing import image

In [ ]:
size = 512
bat_size = 16
split = 0.31
epoch = 10

In [ ]:
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=2,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=[0.0, 0.2],
    shear_range=0.0,
    zoom_range=0.1,
    channel_shift_range=0.0,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip = True,
    vertical_flip = True,
    validation_split=split,
)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    train,
    directory=train_path,
    x_col="image_id",
    y_col="label",
    weight_col=None,
    target_size=(size, size),
    color_mode="rgb",
    classes=None,
    class_mode="categorical",
    batch_size=bat_size,
    shuffle=True,
    seed=None,
    subset='training',
    #save_to_dir=None,
    #save_prefix="",
    #save_format="jpg",
    #subset=None,
    interpolation="nearest",
    validate_filenames=True
)

In [ ]:
validation_generator = train_datagen.flow_from_dataframe(
    train,
    directory=train_path,
    x_col="image_id",
    y_col="label",
    weight_col=None,
    target_size=(size, size),
    color_mode="rgb",
    classes=None,
    class_mode="categorical",
    batch_size=bat_size,
    subset='validation',
    shuffle=True,
    seed=None,
    #save_to_dir=None,
    #save_prefix="",
    #save_format="png",
    #subset=None,
    #interpolation="nearest",
    validate_filenames=True
)

In [ ]:
# -*- coding: utf-8 -*-
from keras.models import Sequential
#initialize nn
import keras
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense
import tensorflow as tf
from keras.layers import BatchNormalization
from keras.layers import Dropout

#basic cnn
model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(size,size, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
    bias_regularizer=tf.keras.regularizers.l2(1e-4),
    activity_regularizer=tf.keras.regularizers.l2(1e-5)))
model.add(Dense(256, activation='relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
    bias_regularizer=tf.keras.regularizers.l2(1e-4),
    activity_regularizer=tf.keras.regularizers.l2(1e-5)))
model.add(Dropout(0.25))
model.add(Dense(5, activation = 'softmax'))

opt = keras.optimizers.Adam(learning_rate=0.001)
#opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
#opt = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_acc",
    min_delta=0.80,
    patience=2,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

In [ ]:
'''model.fit_generator(train_generator,
                         epochs = epoch,
                         validation_data = validation_generator,
                         verbose=1,
                         callbacks = [callback])'''

# EfficientNet B7

In [ ]:
from tensorflow import keras
from tensorflow.keras.applications import EfficientNetB7


model = EfficientNetB7(include_top=False, weights="imagenet", input_shape=(512, 512, 3))

# Freeze the pretrained weights
model.trainable = False

inputs = keras.Input(shape=(512, 512, 3))
x = model(inputs, training=False)
#x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Flatten()(x)
#x = keras.layers.Dense(64, activation='relu')(x)
#x = keras.layers.Dropout(0.2)(x)
#x = keras.layers.Dense(256, activation='relu')(x)
#x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(5, activation='softmax')(x)
model = keras.Model(inputs, outputs)

#x = Flatten()(model.layers[-1].output)
#x = Dense(1024, activation='relu')(x)
#outputs = layers.Dense(5, activation="softmax")(x)

# Compile
#model = tf.keras.Model(model.inputs, outputs, name="EfficientNet")
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)

In [ ]:
model.summary()

In [ ]:
'''model.fit(train_generator,
         epochs = epoch,
         validation_data = validation_generator,
         verbose=1,
         callbacks = [callback])'''

# MobileNet

In [ ]:
!pip install --upgrade tensorflow_hub

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf

In [ ]:
m = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v1_100_224/classification/4"),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
m.build([None, 512, 512, 3])# Batch input shape.

In [ ]:
m.summary()

In [ ]:
m.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
m.fit(train_generator,
     epochs = epoch,
     validation_data = validation_generator,
     verbose=1)

In [ ]:
import numpy as np
preds = []
ss = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

for image in ss.image_id:
    img = keras.preprocessing.image.load_img('../input/cassava-leaf-disease-classification/test_images/' + image)
    img = keras.preprocessing.image.img_to_array(img)
    img = keras.preprocessing.image.smart_resize(img, (size, size))
    img = np.expand_dims(img, 0)
    prediction = model.predict(img)
    preds.append(np.argmax(prediction))

my_submission = pd.DataFrame({'image_id': ss.image_id, 'label': preds})
my_submission.to_csv('submission.csv', index=False) 

In [ ]:
my_submission

In [ ]:
#model.save('model.h5')

In [ ]:
m.save('mobile.h5')